<a href="https://colab.research.google.com/github/mariaeduardagimenes/Spark/blob/master/Spark_socket_to_file_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exemplo Para Leitura Via Socket e Gravação em Arquivo**

In [ ]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.functions import col


spark = SparkSession \
    .builder \
    .appName("StructuredWorkWordCountToFile") \
    .getOrCreate()

In [ ]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .option('includeTimestamp', 'true')\
    .load()

# Divide as linhas recebidas em cada palavra
words = lines.select(
        explode(split(lines.value, ' ')).alias('word'),
        col("timestamp")#lines.timestamp
    )

# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
windowedCounts = words \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(col("timestamp"), "10 minutes", "5 minutes"),
        words.word) \
    .count()

In [ ]:
#confere o tipo de dataframe
windowedCounts.isStreaming

True

SyntaxError: invalid syntax (<ipython-input-16-d8bfb774be46>, line 1)

In [ ]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = windowedCounts\
    .writeStream\
    .outputMode('append')\
    .format('json')\
    .option('truncate', 'false')\
    .option("path", "/mariaeduardagimenes/output_csv")\
    .option("checkpointLocation", "/mariaeduardagimenes/output_csv/chekPoint")\
    .start()



query.awaitTermination() #aguarda até que a "streaming query" termine 

Py4JJavaError: An error occurred while calling o106.start.
: java.io.IOException: mkdir of /mariaeduardagimenes/output_csv/_spark_metadata failed
	at org.apache.hadoop.fs.FileSystem.primitiveMkdir(FileSystem.java:1067)
	at org.apache.hadoop.fs.DelegateToFileSystem.mkdir(DelegateToFileSystem.java:183)
	at org.apache.hadoop.fs.FilterFs.mkdir(FilterFs.java:201)
	at org.apache.hadoop.fs.FileContext$4.next(FileContext.java:730)
	at org.apache.hadoop.fs.FileContext$4.next(FileContext.java:726)
	at org.apache.hadoop.fs.FSLinkResolver.resolve(FSLinkResolver.java:90)
	at org.apache.hadoop.fs.FileContext.mkdir(FileContext.java:733)
	at org.apache.spark.sql.execution.streaming.FileContextBasedCheckpointFileManager.mkdirs(CheckpointFileManager.scala:303)
	at org.apache.spark.sql.execution.streaming.HDFSMetadataLog.<init>(HDFSMetadataLog.scala:66)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.<init>(CompactibleFileStreamLog.scala:47)
	at org.apache.spark.sql.execution.stream

Para rodar é necessário ter o "nc" instalado e utiliar o comando antes de iniciar a célula anterior

nc -lk 9999